In [ ]:
# Mounting the Google Drive and listing the Contents

from google.colab import drive
drive.mount('/content/gdrive')
%cd /content/gdrive/My Drive/DL project_aug1/DL project
!ls

# we use tensor flow and Keras Model. Importing the libraries
%tensorflow_version 2.x

from keras.models import Sequential
from keras.layers import Activation, GlobalAveragePooling2D, Dropout, Dense, Flatten
from keras.optimizers import Adam
from keras.callbacks import TensorBoard, EarlyStopping
from sklearn.metrics import classification_report
from keras.applications import resnet
from keras.layers import Input
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras import Model
import numpy as np
# We are applying early stopping if value loss increased for 5 approaches
tensorboard = TensorBoard(log_dir='./ResNet50', histogram_freq=0, write_graph=True, write_images=False)
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=5)

# Setting the image sizes t0 224
image_size = 224
IMG_SHAPE = (image_size, image_size, 3)  # Applying image shape as 224,224,3
base_model = resnet.ResNet50(weights='imagenet', include_top=False, input_shape=IMG_SHAPE)
base_model.summary()

# Making the model that contain 4 classes
CLASSES = 4
x = base_model.output
x = GlobalAveragePooling2D(name='avg_pool')(x)
x = Dropout(0.4)(x)       # Applying dropout as 0.4 or 40%
predictions = Dense(CLASSES, activation='softmax')(x) # Dense layers are activated using softmax
model = Model(inputs=base_model.input, outputs=predictions)

for layer in base_model.layers:
    layer.trainable = True

model.compile(loss='sparse_categorical_crossentropy', optimizer="adam", metrics=['accuracy'])

# Preparing the data argumentation using image data generator
train_datagen = ImageDataGenerator(rescale=0.2,
                                   rotation_range=30,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True,
                                   validation_split=0.2)

train_generator = train_datagen.flow_from_directory('/content/gdrive/My Drive/DL project_aug1/DL project/training',
                                                    target_size=(image_size, image_size),
                                                    batch_size=32,        #Applying a batch size of 32 for training
                                                    class_mode='sparse',
                                                    subset='training')

validation_generator = train_datagen.flow_from_directory('/content/gdrive/My Drive/DL project_aug1/DL project/testing',
                                                         target_size=(image_size, image_size),
                                                         batch_size=32,
                                                         class_mode='sparse',
                                                         subset='validation')

# Fitting the model For training  and validation generated images
history = model.fit(train_generator,
                    steps_per_epoch=train_generator.samples // train_generator.batch_size,
                    validation_data=validation_generator,
                    validation_steps=validation_generator.samples // validation_generator.batch_size,
                    epochs=7,
                    callbacks=[tensorboard, es])

# Evaluating the model
scores = model.evaluate(validation_generator, verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

# Generating the classification report
y_pred = model.predict(validation_generator, batch_size=64, verbose=1)
y_pred_bool = np.argmax(y_pred, axis=1)
print(classification_report(validation_generator.classes, y_pred_bool, target_names=train_generator.class_indices.keys()))

# Saving of the model along with accuracy
model.save("ResNet50accuracy87.h5")

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
/content/gdrive/My Drive/DL project_aug1/DL project
ResNet50	       testing	 X_test.pickle	 Y_test.pickle
ResNet50accuracy87.h5  training  X_train.pickle  Y_train.pickle
Colab only includes TensorFlow 2.x; %tensorflow_version has no effect.


Model: "resnet50"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_4             │ (None, 224, 224, 3)    │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1_pad (ZeroPadding2D) │ (None, 230, 230, 3)    │              0 │ input_layer_4[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1_conv (Conv2D)       │ (None, 112, 112, 64)   │          9,472 │ conv1_pad[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1_bn                  │ (None, 112, 112, 64)   │            256 │ conv1_conv[0][0]       │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1_relu (Activation)   │ (None, 112, 112, 64)   │              0 │ conv1_bn[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ pool1_pad (ZeroPadding2D) │ (None, 114, 114, 64)   │              0 │ conv1_relu[0][0]       │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ pool1_pool (MaxPooling2D) │ (None, 56, 56, 64)     │              0 │ pool1_pad[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_1_conv       │ (None, 56, 56, 64)     │          4,160 │ pool1_pool[0][0]       │
│ (Conv2D)                  │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_1_bn         │ (None, 56, 56, 64)     │            256 │ conv2_block1_1_conv[0… │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_1_relu       │ (None, 56, 56, 64)     │              0 │ conv2_block1_1_bn[0][… │
│ (Activation)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_2_conv       │ (None, 56, 56, 64)     │         36,928 │ conv2_block1_1_relu[0… │
│ (Conv2D)                  │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_2_bn         │ (None, 56, 56, 64)     │            256 │ conv2_block1_2_conv[0… │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_2_relu       │ (None, 56, 56, 64)     │              0 │ conv2_block1_2_bn[0][… │
│ (Activation)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_0_conv       │ (None, 56, 56, 256)    │         16,640 │ pool1_pool[0][0]       │
│ (Conv2D)                  │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_3_conv       │ (None, 56, 56, 256)    │         16,640 │ conv2_block1_2_relu[0… │
│ (Conv2D)             

 Total params: 23,587,712 (89.98 MB)

 Trainable params: 23,534,592 (89.78 MB)

...

45/45 ━━━━━━━━━━━━━━━━━━━━ 1197s 25s/step - accuracy: 0.6914 - loss: 0.9032 - val_accuracy: 0.2969 - val_loss: 44195.8594
Epoch 2/7
 1/45 ━━━━━━━━━━━━━━━━━━━━ 18:53 26s/step - accuracy: 0.8125 - loss: 0.5243

/usr/lib/python3.10/contextlib.py:153: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


45/45 ━━━━━━━━━━━━━━━━━━━━ 29s 67ms/step - accuracy: 0.8125 - loss: 0.5243 - val_accuracy: 0.1429 - val_loss: 27398.5410
Epoch 3/7
45/45 ━━━━━━━━━━━━━━━━━━━━ 1181s 25s/step - accuracy: 0.7972 - loss: 0.5026 - val_accuracy: 0.2812 - val_loss: 745.1809
Epoch 4/7
45/45 ━━━━━━━━━━━━━━━━━━━━ 28s 46ms/step - accuracy: 0.9688 - loss: 0.1701 - val_accuracy: 0.3571 - val_loss: 533.6638
Epoch 5/7
45/45 ━━━━━━━━━━━━━━━━━━━━ 1132s 25s/step - accuracy: 0.8794 - loss: 0.2789 - val_accuracy: 0.3125 - val_loss: 157.8649
Epoch 6/7
45/45 ━━━━━━━━━━━━━━━━━━━━ 27s 46ms/step - accuracy: 0.7812 - loss: 0.8208 - val_accuracy: 0.2143 - val_loss: 157.8633
Epoch 7/7
45/45 ━━━━━━━━━━━━━━━━━━━━ 1152s 25s/step - accuracy: 0.8332 - loss: 0.4579 - val_accuracy: 0.2500 - val_loss: 2864.5493
3/3 ━━━━━━━━━━━━━━━━━━━━ 14s 3s/step - accuracy: 0.2376 - loss: 2367.6594
Test loss: 2412.608154296875
Test accuracy: 0.25641027092933655
3/3 ━━━━━━━━━━━━━━━━━━━━ 20s 6s/step


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


              precision    recall  f1-score   support

      glioma       0.26      1.00      0.42        20
  meningioma       0.00      0.00      0.00        23
    no_tumor       0.00      0.00      0.00        21
   pituitary       0.00      0.00      0.00        14

    accuracy                           0.26        78
   macro avg       0.07      0.25      0.10        78
weighted avg       0.07      0.26      0.11        78



In [ ]:


from google.colab import drive
drive.mount('/content/gdrive')
%cd /content/gdrive/My Drive/DL project_aug1/DL project
!ls

import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Activation, GlobalAveragePooling2D, Dropout, Dense, Flatten, BatchNormalization, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import TensorBoard, EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import classification_report
from tensorflow.keras.applications import ResNet50
import numpy as np

tensorboard = TensorBoard(log_dir='./ResNet50', histogram_freq=0, write_graph=True, write_images=False)
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=5)
lr_reduction = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, min_lr=0.00001)

image_size = 224
IMG_SHAPE = (image_size, image_size, 3)
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=IMG_SHAPE)
base_model.summary()

CLASSES = 4
x = base_model.output
x = GlobalAveragePooling2D(name='avg_pool')(x)
x = Dropout(0.4)(x)
x = BatchNormalization()(x)  # Add Batch Normalization
predictions = Dense(CLASSES, kernel_regularizer=tf.keras.regularizers.l2(0.01), activation='softmax')(x)  # Add L2 regularization

model = Model(inputs=base_model.input, outputs=predictions)

for layer in base_model.layers:
    layer.trainable = True

model.compile(loss='sparse_categorical_crossentropy', optimizer=Adam(learning_rate=0.001), metrics=['accuracy'])

# Prepare the data
train_datagen = ImageDataGenerator(rescale=0.2,
                                   rotation_range=40,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True,
                                   fill_mode='nearest',
                                   validation_split=0.2)

train_generator = train_datagen.flow_from_directory('/content/gdrive/My Drive/DL project_aug1/DL project/training',
                                                    target_size=(image_size, image_size),
                                                    batch_size=32,
                                                    class_mode='sparse',
                                                    subset='training')

validation_generator = train_datagen.flow_from_directory('/content/gdrive/My Drive/DL project_aug1/DL project/testing',
                                                         target_size=(image_size, image_size),
                                                         batch_size=32,
                                                         class_mode='sparse',
                                                         subset='validation')

# Fit the model
history = model.fit(train_generator,
                    steps_per_epoch=train_generator.samples // train_generator.batch_size,
                    validation_data=validation_generator,
                    validation_steps=validation_generator.samples // validation_generator.batch_size,
                    epochs=10,
                    callbacks=[tensorboard, es, lr_reduction])

# Evaluate the model
scores = model.evaluate(validation_generator, verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

# Generate classification report
y_pred = model.predict(validation_generator, batch_size=64, verbose=1)
y_pred_bool = np.argmax(y_pred, axis=1)
print(classification_report(validation_generator.classes, y_pred_bool, target_names=train_generator.class_indices.keys()))

# Save the model
model.save("ResNet50_improved.h5")


Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
/content/gdrive/My Drive/DL project_aug1/DL project
ResNet50	       testing	 ViT		X_train.pickle	Y_train.pickle
ResNet50accuracy87.h5  training  X_test.pickle	Y_test.pickle


Model: "resnet50"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_5             │ (None, 224, 224, 3)    │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1_pad (ZeroPadding2D) │ (None, 230, 230, 3)    │              0 │ input_layer_5[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1_conv (Conv2D)       │ (None, 112, 112, 64)   │          9,472 │ conv1_pad[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1_bn                  │ (None, 112, 112, 64)   │            256 │ conv1_conv[0][0]       │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1_relu (Activation)   │ (None, 112, 112, 64)   │              0 │ conv1_bn[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ pool1_pad (ZeroPadding2D) │ (None, 114, 114, 64)   │              0 │ conv1_relu[0][0]       │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ pool1_pool (MaxPooling2D) │ (None, 56, 56, 64)     │              0 │ pool1_pad[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_1_conv       │ (None, 56, 56, 64)     │          4,160 │ pool1_pool[0][0]       │
│ (Conv2D)                  │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_1_bn         │ (None, 56, 56, 64)     │            256 │ conv2_block1_1_conv[0… │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_1_relu       │ (None, 56, 56, 64)     │              0 │ conv2_block1_1_bn[0][… │
│ (Activation)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_2_conv       │ (None, 56, 56, 64)     │         36,928 │ conv2_block1_1_relu[0… │
│ (Conv2D)                  │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_2_bn         │ (None, 56, 56, 64)     │            256 │ conv2_block1_2_conv[0… │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_2_relu       │ (None, 56, 56, 64)     │              0 │ conv2_block1_2_bn[0][… │
│ (Activation)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_0_conv       │ (None, 56, 56, 256)    │         16,640 │ pool1_pool[0][0]       │
│ (Conv2D)                  │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_3_conv       │ (None, 56, 56, 256)    │         16,640 │ conv2_block1_2_relu[0… │
│ (Conv2D)             

 Total params: 23,587,712 (89.98 MB)

 Trainable params: 23,534,592 (89.78 MB)

...

/usr/lib/python3.10/contextlib.py:153: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


71/71 ━━━━━━━━━━━━━━━━━━━━ 28s 34ms/step - accuracy: 0.8438 - loss: 0.6255 - val_accuracy: 0.2857 - val_loss: 2866.7722 - learning_rate: 0.0010
Epoch 3/10
71/71 ━━━━━━━━━━━━━━━━━━━━ 1919s 26s/step - accuracy: 0.7773 - loss: 0.7321 - val_accuracy: 0.3281 - val_loss: 1.4056 - learning_rate: 0.0010
Epoch 4/10
71/71 ━━━━━━━━━━━━━━━━━━━━ 29s 38ms/step - accuracy: 0.8438 - loss: 0.6825 - val_accuracy: 0.3571 - val_loss: 18.4599 - learning_rate: 0.0010
Epoch 5/10
71/71 ━━━━━━━━━━━━━━━━━━━━ 1852s 26s/step - accuracy: 0.7800 - loss: 0.6682 - val_accuracy: 0.4531 - val_loss: 1.6133 - learning_rate: 0.0010
Epoch 6/10
71/71 ━━━━━━━━━━━━━━━━━━━━ 29s 48ms/step - accuracy: 0.7500 - loss: 0.7427 - val_accuracy: 0.2857 - val_loss: 2.2041 - learning_rate: 0.0010
Epoch 7/10
71/71 ━━━━━━━━━━━━━━━━━━━━ 1859s 26s/step - accuracy: 0.8378 - loss: 0.4872 - val_accuracy: 0.4688 - val_loss: 1.4978 - learning_rate: 2.0000e-04
Epoch 8/10
71/71 ━━━━━━━━━━━━━━━━━━━━ 28s 63ms/step - accuracy: 0.8438 - loss: 0.5754 - 

2/3 ━━━━━━━━━━━━━━━━━━━━ 5s 5s/step  

3/3 ━━━━━━━━━━━━━━━━━━━━ 20s 5s/step


              precision    recall  f1-score   support

      glioma       1.00      0.05      0.10        20
  meningioma       0.32      0.57      0.41        23
    no_tumor       0.33      0.52      0.41        21
   pituitary       0.33      0.07      0.12        14

    accuracy                           0.33        78
   macro avg       0.50      0.30      0.26        78
weighted avg       0.50      0.33      0.28        78

